**Idea Generator**

Adapted from paraschopra/generating-text-small-corpus

By XDGFX

In [1]:
from fastai import *
from fastai.text import * 

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
# Patent data from "daily patent assignment text for 10/18/2016 - US Patent and Trademark Office"
# View patent-info for information on patent datasets

df = pd.read_csv('patent-data-extended-40.csv', header=None, names=['idea'])

In [5]:
df['idea'] = df['idea'].str.lower()

In [6]:
df.head()

,idea
0,abap channels for event signaling
1,abap unified connectivity
2,abbreviation handling in web search
3,abdominal wall treatment devices
4,abist address generation


In [7]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [8]:
len(df)

31687

In [9]:
len(train_df)

30102

In [10]:
len(valid_df)

1585

In [11]:
nan_rows = df[df['idea'].isnull()]

In [12]:
nan_rows

,idea


In [13]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='idea')

In [14]:
#data_clas = TextClasDataBunch.from_df('data', train_df, valid_df, text_cols='text', label_cols='author', vocab=data_lm.train_ds.vocab, bs=32)

In [15]:
learn = language_model_learner(data_lm, arch = AWD_LSTM, pretrained=True, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.547885,5.616721,0.243638,02:20


In [16]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.483150,5.248180,0.272768,03:43


In [17]:
wd=1e-7
lr=1e-3
lrs = lr

In [18]:
learn.fit(15,lrs, wd)

epoch,train_loss,valid_loss,accuracy,time
0,5.220549,5.067709,0.282254,03:44
1,5.074685,4.979054,0.285045,03:44
2,4.938612,4.920932,0.290067,03:43
3,4.805503,4.879516,0.292299,03:54
4,4.675024,4.869472,0.292969,03:44
5,4.533024,4.867552,0.297321,03:43
6,4.392333,4.865705,0.293080,03:43
7,4.249049,4.908504,0.293415,03:40
8,4.112404,4.929600,0.294531,03:44
9,4.002357,4.955546,0.295201,04:05


In [ ]:
for x in range(20): # How many ideas to generate (in thousands)

    all_ideas = []

    idea = learn.predict("xxbos xxfld 1", n_words=5000, temperature=0.8)
    ideas = idea.split("xxbos")
    
    for idea in ideas:
        idea = idea.replace("xxbos xxfld 1 ","").strip()
        if(idea):
            all_ideas.append(idea)

    del all_ideas[0]
    del all_ideas[len(all_ideas) - 1]

    with open("output.csv", "a") as fp:
        fp.write("\n".join(all_ideas))

In [21]:
learn.save_encoder('ml_ft_enc')

In [22]:
train_df.to_pickle('data/ml_train_df.pkl')

In [23]:
valid_df.to_pickle('data/ml_valid_df.pkl')